In [11]:
import h3
from shapely.geometry import Polygon
import folium
import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import geopandas as gpd
import pandas as pd

# Resolution 7 ~ ~1 km hex, resolution 8 ~ 200–300 m
RESOLUTION = 7
FILE_NAME = 'hokkaido.csv'



In [12]:
pokestops_jp = pd.read_csv(FILE_NAME)


In [13]:
# generate h3_index
pokestops_jp['h3_index'] = pokestops_jp.apply(
    lambda row: h3.latlng_to_cell(row['Fort_Latitude'], row['Fort_Longitude'], RESOLUTION),
    axis=1
)

# Count number of spins per hex
h3_counts = pokestops_jp.groupby('h3_index').size().reset_index(name='spin_count')

# Count unique Fort_ID per hex
unique_forts = (
    pokestops_jp.groupby('h3_index')['fort_id']
    .nunique()
    .reset_index(name='pokestop_count')
)

# Merge with spin counts
h3_counts = h3_counts.merge(unique_forts, on='h3_index')

In [14]:
# Convert each h3_index into its corresponding hexagon geometry
def h3_to_polygon(h3_index):
    boundary = h3.cell_to_boundary(h3_index)  # returns list of (lat, lon)
    return Polygon([(lon, lat) for lat, lon in boundary])  # note: shapely uses (x, y) = (lon, lat)

h3_counts['geometry'] = h3_counts['h3_index'].apply(h3_to_polygon)


# Convert into a GeoDataFrame
gdf = gpd.GeoDataFrame(h3_counts, geometry='geometry', crs='EPSG:4326')


# All geographic data uses WGS84 (EPSG:4326) for storage and H3 indexing, 
# and is reprojected to Web Mercator (EPSG:3857) for map visualisation.

In [21]:
# Create combined metric
gdf['spins_per_stop'] = gdf['spin_count']/gdf['pokestop_count'].round(2)

# Center map
center_lat = pokestops_jp['Fort_Latitude'].mean()
center_lon = pokestops_jp['Fort_Longitude'].mean()

# Create map
m = folium.Map(location=[center_lat, center_lon], zoom_start=11, tiles='OpenStreetMap')

# Maximums for normalization
max_spin = gdf['spin_count'].max()
max_pokestop = gdf['pokestop_count'].max()
max_combined = gdf['spins_per_stop'].max()

# Choose pokestop_count as metric to visualize
metric = 'pokestop_count'
max_val = max_pokestop

# Add hex polygons
for _, row in gdf.iterrows():
    geo_json = gpd.GeoSeries([row['geometry']]).__geo_interface__

    # Log scale normalization for skewed data
    norm_val = np.log1p(row[metric]) / np.log1p(max_val)
    color_hex = colors.to_hex(plt.cm.OrRd(norm_val))

    tooltip_text = (
        f"Spins: {row['spin_count']}<br>"
        f"PokéStops: {row['pokestop_count']}<br>"
        f"Spins per Stop: {row['spins_per_stop']:.2f}"
    )

    folium.GeoJson(
        geo_json,
        style_function=lambda x, fill=color_hex: {
            'fillColor': fill,
            'color': 'grey',
            'weight': 0.5,
            'fillOpacity': 0.6
        },
        tooltip=folium.Tooltip(tooltip_text, sticky=True)
    ).add_to(m)

# Add a title
title_html = """
<h3 align="center" style="font-size:16px">
PokéStop Spins Hex Heatmap<br>
<span style="font-size:10px; font-weight:normal;">
Darker hexes indicate higher number of spins
</span>
</h3>
"""
m.get_root().html.add_child(folium.Element(title_html))

m